<a href="https://colab.research.google.com/github/Tom-Jung/Tom-Jung/blob/main/API_DART_F_%EC%83%81%EC%9E%A5%ED%9A%8C%EC%82%AC_%EC%A4%91%EB%B3%B5%EA%B3%84%EC%A0%95%EA%B3%BC%EB%AA%A9%EC%A0%9C%EA%B1%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install dart-fss

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.7/146.7 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 161.7/161.7 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 8.9 MB/s eta 0:00:00
  Attempting uninstall: termcolor
    Found existing installation: termcolor 3.1.0
    Uninstalling termcolor-3.1.0:
      Successfully uninstalled termcolor-3.1.0


In [2]:
import requests
import pandas as pd
import time

# DART API 인증키 (본인의 API 키 입력)
API_KEY = 'e5ebd99a0228714b70206b955cb7931ffc873783'

# DART 재무제표 API URL
FS_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

# 기업 리스트 엑셀 파일 읽기
file_path = "./corp_list_split.xlsx"  # 로컬 파일 경로
corporations_df = pd.read_excel(file_path, sheet_name="L1", dtype={'code': str})

# 데이터 저장을 위한 리스트
financial_data = []

# 자본변동표 관련 계정과목 필터링 리스트
exclude_accounts = [
    "자본금 변동", "기타포괄손익누계액", "이익잉여금 변동",
    "자본총계 변동", "기타자본항목 변동", "배당금 지급", "주식발행", "자본조정"
]

# 자본 관련 항목 리스트 (우선 재무상태표 항목을 우선하고, 없는 경우 첫 번째로 나온 항목 선택)
capital_related_accounts = ["자본총계", "기초자본", "자본금", "이익잉여금", "기타자본항목"]

# 기업별 재무제표 요청
for _, row in corporations_df.iterrows():
    corp_code = row['code']  # 기업 코드
    corp_name = row.get('name', '알 수 없음')  # 기업 이름

    params = {
        'crtfc_key': API_KEY,
        'corp_code': corp_code,
        'bsns_year': 2023,  # 연도
        'reprt_code': '11011',  # 사업보고서 (연결 재무제표)
        'fs_div': 'OFS'  # 개별 재무제표 (자본변동표 포함 X)
    }

    try:
        # API 요청
        response = requests.get(FS_URL, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()

        if data.get('status') == '000':  # 정상 응답
            if 'list' in data and data['list']:
                unique_accounts = set()  # 중복 방지를 위한 set
                first_appearance_accounts = []  # 첫 번째로 나온 항목을 저장할 리스트

                for item in data['list']:
                    account_name = item['account_nm'].strip()
                    account_id = item.get('account_id', '데이터 없음')  # account_id 추가

                    # 자본변동표 관련 항목 제외
                    if any(excl in account_name for excl in exclude_accounts):
                        continue

                    # 자본 관련 계정 처리: 재무상태표에 있는 항목을 우선하고, 없으면 첫 번째 항목 사용
                    if any(account_name.startswith(capital_account) for capital_account in capital_related_accounts):
                        # 재무상태표에 있는 자본 항목을 우선 처리
                        if account_name not in unique_accounts:
                            unique_accounts.add(account_name)
                            financial_data.append({
                                "기업 이름": corp_name,
                                "기업 코드": corp_code,
                                "연도": 2023,
                                "계정과목명": account_name,
                                "account_id": account_id,  # account_id 추가
                                "금액": item.get('thstrm_amount', '데이터 없음'),  # 2023년 금액
                                "2022년 금액": item.get('frmtrm_amount', '데이터 없음'),  # 2022년 금액
                                "2021년 금액": item.get('lfy_amount', '데이터 없음')  # 2021년 금액
                            })
                    else:
                        # 첫 번째로 나온 자본 관련 항목을 선택하여 추가
                        if account_name not in first_appearance_accounts:
                            first_appearance_accounts.append(account_name)
                            financial_data.append({
                                "기업 이름": corp_name,
                                "기업 코드": corp_code,
                                "연도": 2023,
                                "계정과목명": account_name,
                                "account_id": account_id,  # account_id 추가
                                "금액": item.get('thstrm_amount', '데이터 없음'),  # 2023년 금액
                                "2022년 금액": item.get('frmtrm_amount', '데이터 없음'),  # 2022년 금액
                                "2021년 금액": item.get('lfy_amount', '데이터 없음')  # 2021년 금액
                            })

                print(f"✅ {corp_name}({corp_code}) - 데이터 수집 완료")
            else:
                print(f"⚠️ {corp_name}({corp_code}) - 재무 데이터 없음")
        else:
            print(f"❌ {corp_name}({corp_code}) - API 오류: {data.get('message', '알 수 없는 오류')}")

    except requests.exceptions.RequestException as e:
        print(f"🚨 {corp_name}({corp_code}) - 요청 실패: {e}")

    time.sleep(1)  # API 요청 간격 조정

# DataFrame 변환
df_1 = pd.DataFrame(financial_data)

# 결과 저장
output_excel = "기업_재무제표_2023년_L1.xlsx"

df_1.to_excel(output_excel, index=False)

print(f"✅ 데이터 저장 완료: {output_excel}, {output_excel}")

FileNotFoundError: [Errno 2] No such file or directory: './corp_list_split.xlsx'

#
:------------------------------------------------------------------------------L2------------------------------------

In [ ]:
import requests
import pandas as pd
import time

# DART API 인증키 (본인의 API 키 입력)
API_KEY = 'e5ebd99a0228714b70206b955cb7931ffc873783'

# DART 재무제표 API URL
FS_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

# 기업 리스트 엑셀 파일 읽기
file_path = "./corp_list_split.xlsx"  # 로컬 파일 경로
corporations_df = pd.read_excel(file_path, sheet_name="L2", dtype={'code': str})

# 데이터 저장을 위한 리스트
financial_data = []

# 자본변동표 관련 계정과목 필터링 리스트
exclude_accounts = [
    "자본금 변동", "기타포괄손익누계액", "이익잉여금 변동",
    "자본총계 변동", "기타자본항목 변동", "배당금 지급", "주식발행", "자본조정"
]

# 자본 관련 항목 리스트 (우선 재무상태표 항목을 우선하고, 없는 경우 첫 번째로 나온 항목 선택)
capital_related_accounts = ["자본총계", "기초자본", "자본금", "이익잉여금", "기타자본항목"]

# 기업별 재무제표 요청
for _, row in corporations_df.iterrows():
    corp_code = row['code']  # 기업 코드
    corp_name = row.get('name', '알 수 없음')  # 기업 이름

    params = {
        'crtfc_key': API_KEY,
        'corp_code': corp_code,
        'bsns_year': 2023,  # 연도
        'reprt_code': '11011',  # 사업보고서 (연결 재무제표)
        'fs_div': 'OFS'  # 개별 재무제표 (자본변동표 포함 X)
    }

    try:
        # API 요청
        response = requests.get(FS_URL, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()

        if data.get('status') == '000':  # 정상 응답
            if 'list' in data and data['list']:
                unique_accounts = set()  # 중복 방지를 위한 set
                first_appearance_accounts = []  # 첫 번째로 나온 항목을 저장할 리스트

                for item in data['list']:
                    account_name = item['account_nm'].strip()
                    account_id = item.get('account_id', '데이터 없음')  # account_id 추가

                    # 자본변동표 관련 항목 제외
                    if any(excl in account_name for excl in exclude_accounts):
                        continue

                    # 자본 관련 계정 처리: 재무상태표에 있는 항목을 우선하고, 없으면 첫 번째 항목 사용
                    if any(account_name.startswith(capital_account) for capital_account in capital_related_accounts):
                        # 재무상태표에 있는 자본 항목을 우선 처리
                        if account_name not in unique_accounts:
                            unique_accounts.add(account_name)
                            financial_data.append({
                                "기업 이름": corp_name,
                                "기업 코드": corp_code,
                                "연도": 2023,
                                "계정과목명": account_name,
                                "account_id": account_id,  # account_id 추가
                                "금액": item.get('thstrm_amount', '데이터 없음'),  # 2023년 금액
                                "2022년 금액": item.get('frmtrm_amount', '데이터 없음'),  # 2022년 금액
                                "2021년 금액": item.get('lfy_amount', '데이터 없음')  # 2021년 금액
                            })
                    else:
                        # 첫 번째로 나온 자본 관련 항목을 선택하여 추가
                        if account_name not in first_appearance_accounts:
                            first_appearance_accounts.append(account_name)
                            financial_data.append({
                                "기업 이름": corp_name,
                                "기업 코드": corp_code,
                                "연도": 2023,
                                "계정과목명": account_name,
                                "account_id": account_id,  # account_id 추가
                                "금액": item.get('thstrm_amount', '데이터 없음'),  # 2023년 금액
                                "2022년 금액": item.get('frmtrm_amount', '데이터 없음'),  # 2022년 금액
                                "2021년 금액": item.get('lfy_amount', '데이터 없음')  # 2021년 금액
                            })

                print(f"✅ {corp_name}({corp_code}) - 데이터 수집 완료")
            else:
                print(f"⚠️ {corp_name}({corp_code}) - 재무 데이터 없음")
        else:
            print(f"❌ {corp_name}({corp_code}) - API 오류: {data.get('message', '알 수 없는 오류')}")

    except requests.exceptions.RequestException as e:
        print(f"🚨 {corp_name}({corp_code}) - 요청 실패: {e}")

    time.sleep(1)  # API 요청 간격 조정

# DataFrame 변환
df_2 = pd.DataFrame(financial_data)

# 결과 저장
output_excel = "기업_재무제표_2023년_L2.xlsx"

df_2.to_excel(output_excel, index=False)

print(f"✅ 데이터 저장 완료: {output_excel}, {output_excel}")

#
:------------------------------------------------------------------------------L3------------------------------------




In [ ]:
import requests
import pandas as pd
import time

# DART API 인증키 (본인의 API 키 입력)
API_KEY = 'e5ebd99a0228714b70206b955cb7931ffc873783'

# DART 재무제표 API URL
FS_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

# 기업 리스트 엑셀 파일 읽기
file_path = "./corp_list_split.xlsx"  # 로컬 파일 경로
corporations_df = pd.read_excel(file_path, sheet_name="L3", dtype={'code': str})

# 데이터 저장을 위한 리스트
financial_data = []

# 자본변동표 관련 계정과목 필터링 리스트
exclude_accounts = [
    "자본금 변동", "기타포괄손익누계액", "이익잉여금 변동",
    "자본총계 변동", "기타자본항목 변동", "배당금 지급", "주식발행", "자본조정"
]

# 자본 관련 항목 리스트 (우선 재무상태표 항목을 우선하고, 없는 경우 첫 번째로 나온 항목 선택)
capital_related_accounts = ["자본총계", "기초자본", "자본금", "이익잉여금", "기타자본항목"]

# 기업별 재무제표 요청
for _, row in corporations_df.iterrows():
    corp_code = row['code']  # 기업 코드
    corp_name = row.get('name', '알 수 없음')  # 기업 이름

    params = {
        'crtfc_key': API_KEY,
        'corp_code': corp_code,
        'bsns_year': 2023,  # 연도
        'reprt_code': '11011',  # 사업보고서 (연결 재무제표)
        'fs_div': 'OFS'  # 개별 재무제표 (자본변동표 포함 X)
    }

    try:
        # API 요청
        response = requests.get(FS_URL, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()

        if data.get('status') == '000':  # 정상 응답
            if 'list' in data and data['list']:
                unique_accounts = set()  # 중복 방지를 위한 set
                first_appearance_accounts = []  # 첫 번째로 나온 항목을 저장할 리스트

                for item in data['list']:
                    account_name = item['account_nm'].strip()
                    account_id = item.get('account_id', '데이터 없음')  # account_id 추가

                    # 자본변동표 관련 항목 제외
                    if any(excl in account_name for excl in exclude_accounts):
                        continue

                    # 자본 관련 계정 처리: 재무상태표에 있는 항목을 우선하고, 없으면 첫 번째 항목 사용
                    if any(account_name.startswith(capital_account) for capital_account in capital_related_accounts):
                        # 재무상태표에 있는 자본 항목을 우선 처리
                        if account_name not in unique_accounts:
                            unique_accounts.add(account_name)
                            financial_data.append({
                                "기업 이름": corp_name,
                                "기업 코드": corp_code,
                                "연도": 2023,
                                "계정과목명": account_name,
                                "account_id": account_id,  # account_id 추가
                                "금액": item.get('thstrm_amount', '데이터 없음'),  # 2023년 금액
                                "2022년 금액": item.get('frmtrm_amount', '데이터 없음'),  # 2022년 금액
                                "2021년 금액": item.get('lfy_amount', '데이터 없음')  # 2021년 금액
                            })
                    else:
                        # 첫 번째로 나온 자본 관련 항목을 선택하여 추가
                        if account_name not in first_appearance_accounts:
                            first_appearance_accounts.append(account_name)
                            financial_data.append({
                                "기업 이름": corp_name,
                                "기업 코드": corp_code,
                                "연도": 2023,
                                "계정과목명": account_name,
                                "account_id": account_id,  # account_id 추가
                                "금액": item.get('thstrm_amount', '데이터 없음'),  # 2023년 금액
                                "2022년 금액": item.get('frmtrm_amount', '데이터 없음'),  # 2022년 금액
                                "2021년 금액": item.get('lfy_amount', '데이터 없음')  # 2021년 금액
                            })

                print(f"✅ {corp_name}({corp_code}) - 데이터 수집 완료")
            else:
                print(f"⚠️ {corp_name}({corp_code}) - 재무 데이터 없음")
        else:
            print(f"❌ {corp_name}({corp_code}) - API 오류: {data.get('message', '알 수 없는 오류')}")

    except requests.exceptions.RequestException as e:
        print(f"🚨 {corp_name}({corp_code}) - 요청 실패: {e}")

    time.sleep(1)  # API 요청 간격 조정

# DataFrame 변환
df_3 = pd.DataFrame(financial_data)

# 결과 저장
output_excel = "기업_재무제표_2023년_L3.xlsx"

df_3.to_excel(output_excel, index=False)

print(f"✅ 데이터 저장 완료: {output_excel}, {output_excel}")

#
:------------------------------------------------------------------------------L4------------------------------------

In [ ]:
import requests
import pandas as pd
import time

# DART API 인증키 (본인의 API 키 입력)
API_KEY = 'e5ebd99a0228714b70206b955cb7931ffc873783'

# DART 재무제표 API URL
FS_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

# 기업 리스트 엑셀 파일 읽기
file_path = "./corp_list_split.xlsx"  # 로컬 파일 경로
corporations_df = pd.read_excel(file_path, sheet_name="L4", dtype={'code': str})

# 데이터 저장을 위한 리스트
financial_data = []

# 자본변동표 관련 계정과목 필터링 리스트
exclude_accounts = [
    "자본금 변동", "기타포괄손익누계액", "이익잉여금 변동",
    "자본총계 변동", "기타자본항목 변동", "배당금 지급", "주식발행", "자본조정"
]

# 자본 관련 항목 리스트 (우선 재무상태표 항목을 우선하고, 없는 경우 첫 번째로 나온 항목 선택)
capital_related_accounts = ["자본총계", "기초자본", "자본금", "이익잉여금", "기타자본항목"]

# 기업별 재무제표 요청
for _, row in corporations_df.iterrows():
    corp_code = row['code']  # 기업 코드
    corp_name = row.get('name', '알 수 없음')  # 기업 이름

    params = {
        'crtfc_key': API_KEY,
        'corp_code': corp_code,
        'bsns_year': 2023,  # 연도
        'reprt_code': '11011',  # 사업보고서 (연결 재무제표)
        'fs_div': 'OFS'  # 개별 재무제표 (자본변동표 포함 X)
    }

    try:
        # API 요청
        response = requests.get(FS_URL, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()

        if data.get('status') == '000':  # 정상 응답
            if 'list' in data and data['list']:
                unique_accounts = set()  # 중복 방지를 위한 set
                first_appearance_accounts = []  # 첫 번째로 나온 항목을 저장할 리스트

                for item in data['list']:
                    account_name = item['account_nm'].strip()
                    account_id = item.get('account_id', '데이터 없음')  # account_id 추가

                    # 자본변동표 관련 항목 제외
                    if any(excl in account_name for excl in exclude_accounts):
                        continue

                    # 자본 관련 계정 처리: 재무상태표에 있는 항목을 우선하고, 없으면 첫 번째 항목 사용
                    if any(account_name.startswith(capital_account) for capital_account in capital_related_accounts):
                        # 재무상태표에 있는 자본 항목을 우선 처리
                        if account_name not in unique_accounts:
                            unique_accounts.add(account_name)
                            financial_data.append({
                                "기업 이름": corp_name,
                                "기업 코드": corp_code,
                                "연도": 2023,
                                "계정과목명": account_name,
                                "account_id": account_id,  # account_id 추가
                                "금액": item.get('thstrm_amount', '데이터 없음'),  # 2023년 금액
                                "2022년 금액": item.get('frmtrm_amount', '데이터 없음'),  # 2022년 금액
                                "2021년 금액": item.get('lfy_amount', '데이터 없음')  # 2021년 금액
                            })
                    else:
                        # 첫 번째로 나온 자본 관련 항목을 선택하여 추가
                        if account_name not in first_appearance_accounts:
                            first_appearance_accounts.append(account_name)
                            financial_data.append({
                                "기업 이름": corp_name,
                                "기업 코드": corp_code,
                                "연도": 2023,
                                "계정과목명": account_name,
                                "account_id": account_id,  # account_id 추가
                                "금액": item.get('thstrm_amount', '데이터 없음'),  # 2023년 금액
                                "2022년 금액": item.get('frmtrm_amount', '데이터 없음'),  # 2022년 금액
                                "2021년 금액": item.get('lfy_amount', '데이터 없음')  # 2021년 금액
                            })

                print(f"✅ {corp_name}({corp_code}) - 데이터 수집 완료")
            else:
                print(f"⚠️ {corp_name}({corp_code}) - 재무 데이터 없음")
        else:
            print(f"❌ {corp_name}({corp_code}) - API 오류: {data.get('message', '알 수 없는 오류')}")

    except requests.exceptions.RequestException as e:
        print(f"🚨 {corp_name}({corp_code}) - 요청 실패: {e}")

    time.sleep(1)  # API 요청 간격 조정

# DataFrame 변환
df_4 = pd.DataFrame(financial_data)

# 결과 저장
output_excel = "기업_재무제표_2023년_L4.xlsx"

df_4.to_excel(output_excel, index=False)

print(f"✅ 데이터 저장 완료: {output_excel}, {output_excel}")

#
:------------------------------------------------------------------------------L5------------------------------------

In [ ]:
import requests
import pandas as pd
import time

# DART API 인증키 (본인의 API 키 입력)
API_KEY = 'e5ebd99a0228714b70206b955cb7931ffc873783'

# DART 재무제표 API URL
FS_URL = "https://opendart.fss.or.kr/api/fnlttSinglAcntAll.json"

# 기업 리스트 엑셀 파일 읽기
file_path = "./corp_list_split.xlsx"  # 로컬 파일 경로
corporations_df = pd.read_excel(file_path, sheet_name="L5", dtype={'code': str})

# 데이터 저장을 위한 리스트
financial_data = []

# 자본변동표 관련 계정과목 필터링 리스트
exclude_accounts = [
    "자본금 변동", "기타포괄손익누계액", "이익잉여금 변동",
    "자본총계 변동", "기타자본항목 변동", "배당금 지급", "주식발행", "자본조정"
]

# 자본 관련 항목 리스트 (우선 재무상태표 항목을 우선하고, 없는 경우 첫 번째로 나온 항목 선택)
capital_related_accounts = ["자본총계", "기초자본", "자본금", "이익잉여금", "기타자본항목"]

# 기업별 재무제표 요청
for _, row in corporations_df.iterrows():
    corp_code = row['code']  # 기업 코드
    corp_name = row.get('name', '알 수 없음')  # 기업 이름

    params = {
        'crtfc_key': API_KEY,
        'corp_code': corp_code,
        'bsns_year': 2023,  # 연도
        'reprt_code': '11011',  # 사업보고서 (연결 재무제표)
        'fs_div': 'OFS'  # 개별 재무제표 (자본변동표 포함 X)
    }

    try:
        # API 요청
        response = requests.get(FS_URL, params=params, timeout=10)
        response.raise_for_status()
        data = response.json()

        if data.get('status') == '000':  # 정상 응답
            if 'list' in data and data['list']:
                unique_accounts = set()  # 중복 방지를 위한 set
                first_appearance_accounts = []  # 첫 번째로 나온 항목을 저장할 리스트

                for item in data['list']:
                    account_name = item['account_nm'].strip()
                    account_id = item.get('account_id', '데이터 없음')  # account_id 추가

                    # 자본변동표 관련 항목 제외
                    if any(excl in account_name for excl in exclude_accounts):
                        continue

                    # 자본 관련 계정 처리: 재무상태표에 있는 항목을 우선하고, 없으면 첫 번째 항목 사용
                    if any(account_name.startswith(capital_account) for capital_account in capital_related_accounts):
                        # 재무상태표에 있는 자본 항목을 우선 처리
                        if account_name not in unique_accounts:
                            unique_accounts.add(account_name)
                            financial_data.append({
                                "기업 이름": corp_name,
                                "기업 코드": corp_code,
                                "연도": 2023,
                                "계정과목명": account_name,
                                "account_id": account_id,  # account_id 추가
                                "금액": item.get('thstrm_amount', '데이터 없음'),  # 2023년 금액
                                "2022년 금액": item.get('frmtrm_amount', '데이터 없음'),  # 2022년 금액
                                "2021년 금액": item.get('lfy_amount', '데이터 없음')  # 2021년 금액
                            })
                    else:
                        # 첫 번째로 나온 자본 관련 항목을 선택하여 추가
                        if account_name not in first_appearance_accounts:
                            first_appearance_accounts.append(account_name)
                            financial_data.append({
                                "기업 이름": corp_name,
                                "기업 코드": corp_code,
                                "연도": 2023,
                                "계정과목명": account_name,
                                "account_id": account_id,  # account_id 추가
                                "금액": item.get('thstrm_amount', '데이터 없음'),  # 2023년 금액
                                "2022년 금액": item.get('frmtrm_amount', '데이터 없음'),  # 2022년 금액
                                "2021년 금액": item.get('lfy_amount', '데이터 없음')  # 2021년 금액
                            })

                print(f"✅ {corp_name}({corp_code}) - 데이터 수집 완료")
            else:
                print(f"⚠️ {corp_name}({corp_code}) - 재무 데이터 없음")
        else:
            print(f"❌ {corp_name}({corp_code}) - API 오류: {data.get('message', '알 수 없는 오류')}")

    except requests.exceptions.RequestException as e:
        print(f"🚨 {corp_name}({corp_code}) - 요청 실패: {e}")

    time.sleep(1)  # API 요청 간격 조정

# DataFrame 변환
df_5 = pd.DataFrame(financial_data)

# 결과 저장
output_excel = "기업_재무제표_2023년_L5.xlsx"

df_5.to_excel(output_excel, index=False)

print(f"✅ 데이터 저장 완료: {output_excel}, {output_excel}")

In [ ]:
df_fn = pd.concat([df_1,df_2,df_3,df_4,df_5])
df_fn.to_excel('상장법인재무제표_중복제거.xlsx')